In [1]:
import pandas as pd
import numpy as np
import function_initial as fn
import function_frac_wind as  ffw
import matplotlib.pyplot as  plt
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates
import plotly.graph_objects as go


In [2]:
df= pd.read_csv('Initial_cleaned_data.csv')

In [3]:
df=df[['Symbol','Date','Open','High','Low','LTP']]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40365 entries, 0 to 40364
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Symbol  40365 non-null  object 
 1   Date    40365 non-null  object 
 2   Open    40365 non-null  float64
 3   High    40365 non-null  float64
 4   Low     40365 non-null  float64
 5   LTP     40365 non-null  float64
dtypes: float64(4), object(2)
memory usage: 1.8+ MB


In [5]:
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40365 entries, 0 to 40364
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Symbol  40365 non-null  object        
 1   Date    40365 non-null  datetime64[ns]
 2   Open    40365 non-null  float64       
 3   High    40365 non-null  float64       
 4   Low     40365 non-null  float64       
 5   LTP     40365 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 1.8+ MB


In [7]:
stock = 'SCB'
start_date  ='2019-01-01'
end_date = '2019-07-30'

In [8]:
df= df[(df['Date'] > start_date ) & (df['Date'] < end_date ) & (df['Symbol']==stock )]

In [9]:
df.reset_index(drop=True, inplace=True) # index is started from 0 such that there is no error in the code below


In [10]:
df

,Symbol,Date,Open,High,Low,LTP
0,SCB,2019-01-10,605.0,610.0,602.0,608.0
1,SCB,2019-01-13,608.0,608.0,586.0,589.0
2,SCB,2019-01-14,589.0,593.0,585.0,590.0
3,SCB,2019-01-15,590.0,592.0,587.0,592.0
4,SCB,2019-01-16,592.0,599.0,590.0,594.0
...,...,...,...,...,...,...
136,SCB,2019-07-03,667.0,675.0,659.0,669.0
137,SCB,2019-07-04,675.0,675.0,662.0,665.0
138,SCB,2019-07-07,665.0,665.0,659.0,661.0
139,SCB,2019-07-08,660.0,660.0,653.0,655.0


In [11]:
#method 1: fractal candlestick pattern
# determine bullish fractal 
def is_support(df,i):  
  cond1 = df['Low'][i] < df['Low'][i-1]   
  cond2 = df['Low'][i] < df['Low'][i+1]   
  cond3 = df['Low'][i+1] < df['Low'][i+2]   
  cond4 = df['Low'][i-1] < df['Low'][i-2]  
  return (cond1 and cond2 and cond3 and cond4) 
# determine bearish fractal
def is_resistance(df,i):  
  cond1 = df['High'][i] > df['High'][i-1]   
  cond2 = df['High'][i] > df['High'][i+1]   
  cond3 = df['High'][i+1] > df['High'][i+2]   
  cond4 = df['High'][i-1] > df['High'][i-2]  
  return (cond1 and cond2 and cond3 and cond4)

# to make sure the new level area does not exist already
def is_far_from_level(value, levels, df):    
  ave =  np.mean(df['High'] - df['Low'])    
  return np.sum([abs(value-level)<ave for _,level in levels])==0

# a list to store resistance and support levels
levels = []
for i in range(2, df.shape[0] - 2):  
  if is_support(df, i):    
    low = df['Low'][i]    
    if is_far_from_level(low, levels, df):      
      levels.append((i, low))  
  elif is_resistance(df, i):    
    high = df['High'][i]    
    if is_far_from_level(high, levels, df):      
      levels.append((i, high))

# for visualization

### didnot work mplfinance

```python
def plot_all(levels, df):    
  fig, ax = plt.subplots(figsize=(16, 9))   
  candlestick_ohlc(ax,df.values,width=0.6, colorup='green', 
    colordown='red', alpha=0.8)    
  date_format = mpl_dates.DateFormatter('%d %b %Y')
  ax.xaxis.set_major_formatter(date_format)    
  for level in levels:        
    plt.hlines(level[1], xmin = df['Date'][level[0]], xmax = 
      max(df['Date']), colors='blue', linestyle='--')    
  fig.show()

'''
  

In [15]:
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['LTP'])])

for level in levels:
    index, value = level  # Unpack the index and value from the tuple
    fig.add_shape(type='line',
                  x0=df['Date'][index],
                  y0=value,
                  x1=df['Date'][index+1],  # Assuming the date is continuous
                  y1=value,
                  line=dict(color='blue', width=1, dash='dash'))


fig.update_layout(title='Candlestick Chart with Levels')


fig.show()



In [16]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                                    open=df['Open'],
                                    high=df['High'],
                                    low=df['Low'],
                                    close=df['LTP'])])

for level in levels:
    index, value = level  # Unpack the index and value from the tuple
    x0 = df['Date'][index]
    y0 = value
    x1 = df['Date'][index + 1] if index + 1 < len(df['Date']) else df['Date'][index]  # Assuming the date is continuous
    y1 = value
    fig.add_shape(type='line',
                  x0=x0,
                  y0=y0,
                  x1=x1,
                  y1=y1,
                  line=dict(color='blue', width=1, dash='dash'))

    # Specify the desired length extension
    length_extension = pd.Timedelta(days=100)
    if index + 1 < len(df['Date']):
        x1_extension = pd.to_datetime(x1) + length_extension
    else:
        x1_extension = pd.to_datetime(x1)

    fig.add_shape(type='line',
                  x0=x1,
                  y0=y1,
                  x1=x1_extension,
                  y1=y1,
                  line=dict(color='blue', width=1, dash='dash'))

fig.update_layout(height=600, width=1200,title = 'Candlestick chart with levels')

fig.show()
